# Recommendation Using TF-IDF weighted Words Embedding

**STEP** <br>
1. Create TF-IDF 
2. Convert a tf-idf dictionary with word as key, idf as a value
3. Get TF-IDF features
4. Combine pretrained words embedding with TF-IDF
5. Calculate Cosine Similarity 
6. Recommend Law 

### Read ckiptagger & Dataframe

In [129]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import datetime
import pickle
from bs4 import BeautifulSoup
import jieba

path = "./ckip/data"
ws = WS(path)

df = pd.read_csv('../data/courses.csv').fillna('')
# Replace '@' with ' ' in original dataframe
#df.token = df.token.apply(lambda text: str(text).replace('@',' '))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


### Import Words Dictionary

### Read Pretrained Words Embedding
詞向量訓練文本來源為中文維基百科，全部的訓練文本可於[此](https://dumps.wikimedia.org/zhwiki/latest/zhwiki-latest-pages-articles.xml.bz2)下載最新版的中文維基百科。<br>
維基百科2014（總詞彙數：655K，400維詞向量，下載大小為2.5G）<br>
來源：[元智大學自然語言處理實驗室](http://nlp.innobic.yzu.edu.tw/demo/word-embedding.html)

In [130]:
token_list = np.load('./ckip/data/embedding_word/token_list.npy')
vector_list = np.load('./ckip/data/embedding_word/vector_list.npy')

embeddings = dict(zip(token_list, vector_list))
d = vector_list.shape[1]

In [131]:
course_df_original = pd.read_csv('../data/courses.csv').fillna('')
course_df = course_df_original.drop(['course_id', 'teacher_id', 'course_published_at_local', 'course_published_at_local', 'course_price'], axis=1)
course_df['description'] = course_df['description'].replace('([\<]).*?([\>])','',regex=True)
course_df = course_df.replace('\n', '',regex=True).replace('&.*;', '',regex=True).replace("--&?", "",regex=True)
course_df = course_df.replace('    ', ' ',regex=True).astype(str)
course_df = course_df.replace('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', ' ',regex=True).astype(str)
course_df.insert(0, 'course_id', course_df_original['course_id'])
course_df[:2]
info = []
for i in range(course_df.shape[0]):
    t = course_df.loc[i, :].values.flatten().tolist()
    info.append([t[0], ' '.join(t[:])])
info = pd.DataFrame(info, columns = ['course_id', 'text'])
info.iloc[33]['text']

'56385f4e423bdd0a00103c3c 怪獸工場 - 色彩光學篇 跟創作成為戀人~永遠的熱戀下去吧!10多年來的教學繪畫經驗分享給喜歡繪畫的朋友們！相關怪獸課程網址 @bluebobo3333?tr=bluebobo3333 藝術,設計 平面設計,色彩學,電腦繪圖 設計_平面設計,藝術_色彩學,藝術_電腦繪圖 photoshop,色彩光學,色彩學 講師簡介-藍國峰(直人Blue)怪獸FB- 、中階學員，所完整規劃的基礎課程。對於想手繪轉電繪、或是會基礎，卻無法快速掌握電繪技巧的你所規劃的課程。在灰階轉彩時，我們會遇到幾種最為常見的狀況，顏色過髒，配色過俗，沒有重點，跟場景的顏色搭不起來等...這些都是大家最為常見的困擾點。其實，方法很簡單，就差懂不懂當中的竅門而已！在此色彩光學篇，將會一步步的帶領大家，解決這些惱人的問題！了解光，探索顏色，完成一幅具有完整度的角色作品。首先我們必須先要了解光跟色彩的關係，當物體接觸到光後，會產生了哪些化學反應，當你了解了光，才能理解色彩運用的時機。當然，色彩心理學的基礎概念，是你一定需要必備的知識武器，首先，會先透過材質球及實例解析及演練，來了解色彩反應與功能操作應用方式。再來進入實際操作的階段，一步步的了解圖層與顏色之間的應用方式，藉由圖層強大的屬性，快速的達到心中理想的色彩變化。透過長年繪畫的經驗及累積，分享一些實用的操作技巧，讓角色或是在整體的情境氛圍掌控，都可以有著大大的加分效果！最後，會以真實的角色，做詳細的解說與拆解動作，讓同學們可以了解所學的技巧，如何予以應用在各種自己的創作中。戰谷松崗科技-馬雅之怒版權所有當我們可以靈活運用灰階及色彩篇所有的功能及觀念後，我們才可以談到所謂的『美感設計』。美感這環，其實是繪圖中最為困難的部分，必須要用主題單元的方式來做教學。大家若喜歡怪獸的教學，之後會以人物、角色、怪物、繪本、場景單元的方式，一一的深入分析及探討，讓大家可以更能找到專屬自己的美術價值。也懇請大家可以多多分享支持，幫怪獸集氣，一起與怪獸變得更強大！灰階上色寫實示範實例 會以完成一個可愛遊戲角色完稿為目標，若能完全了解課程中所提的觀念，是可以運用在各種商業所需的創作上。 Photoshop CS6+數位板 這是一套專門針對初、中階學員，完整規劃的基礎課程。著重於觀念的養成及功能的應用，只要是對電繪有興趣的學員

### Tf-idf for Tokenized Text in Dataframe

In [132]:
# TF-IDF Model
tfidf_ml = TfidfVectorizer(tokenizer=jieba.lcut)
tfidf_ml.fit(info.text)

# TF-IDF Dicitonary
dictionary = dict(zip(tfidf_ml.get_feature_names(), list(tfidf_ml.idf_)))

# feature name
tfidf_feature = tfidf_ml.get_feature_names()
'|'.join(tfidf_feature[5000:5050])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'lv2|lvm|lvmh|lydia|lydian|lymma|lynn|lyric|lyrics|m|m1|m5|m6|m88|ma|mac|macaron|macbook|macd|machine|macincloud|macleod|macos|macosx|macromicro|madame|madaster|made|madera|madker|magazine|maggie|maggiemay|magic|magicphotoshop2018|magnus|magritte|mai|mail|mailchimp|maine|mainly|maintenance|major|mak|make|makee|makeline|makepoint|maker'

### Newly Entered Text Preprocess function
- Remove Punctuation
- Remove Spaces
- Sentence Segment
- turn into list

In [133]:
def Preprocess(text):
    rule = re.compile(r'[^a-zA-Z0-9\u4e00-\u9fa5]')
    text = rule.sub(' ',str(text))
    text = re.sub(' +', '',text)
    #text = ws([text],sentence_segmentation=True, recommend_dictionary=word_dict)
    text_lst = ws([text],sentence_segmentation=True)
    text_lst2 = [x for l in text_lst for x in l if len(x) > 1]
    #text_lst2.append(text)
    text_lst2 = list(set(text_lst2))
    if('更多' in text_lst2):
        text_lst2.remove('更多')
    print(text_lst2)
    return text_lst2

### Calculate TF-IDF Weighted Word Embedding

In [134]:
starttime = datetime.datetime.now()

# TF-IDF weighted Word2Vec
tfidf_text_vect = [] # tfidf-w2v is stored in this list
row = 0

for text in info.text.apply(lambda text: text.split()):
    text_vect = np.zeros(300)
    weight_sum = 0
    for word in text:
        if word in embeddings.keys() and word in tfidf_feature:
            vec = embeddings[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tfidf_text_vect.append(text_vect)
    row += 1

# calculate running time
endtime = datetime.datetime.now()
print("建立模型時間: ",endtime - starttime)

建立模型時間:  0:00:10.651342


### Law Recommendation Function
輸入內文 --> 跑出推薦的前十個相近內文對應的法律

In [135]:
def recommend_law(text, tfidf_text_vect = tfidf_text_vect):
    top_k = 50
    text = Preprocess(text)
    #print(text)
    text_vect = np.zeros(300) # w2v size
    weight_sum = 0
    for word in text:
        if word in embeddings.keys() and word in tfidf_feature:
            vec = embeddings[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tmp_vect = [*tfidf_text_vect,text_vect]
    new_cos_sim = cosine_similarity(tmp_vect, tmp_vect)
    sim_score = np.sort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:top_k+1]
    tmp_top_10_law = info[['text']].iloc[np.argsort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:top_k+1]]
    tmp_top_10_law['similarity_score'] = [round(score*100,1) for score in sim_score]
    return tmp_top_10_law

### Try an Example
輸入內容便可以推薦出適合的法律<br>

In [136]:
starttime = datetime.datetime.now()

newtext = '''
人文＿更多人文
'''
result = recommend_law(newtext)

# calculate running time
endtime = datetime.datetime.now()
print("搜尋推薦時間: ",endtime - starttime)
result


['人文']
搜尋推薦時間:  0:00:00.145921


,text,similarity_score
163,59c1c8c84c36510700f925b2 三小時學會拆解社會現象與新聞議題！ 我是筆...,100.0
198,5aaf2dad1e4ee6001e9632fd 三小時教你怎麼講道德不輸人 我是筆名人渣文...,100.0
702,5fe6fe2c5a5ba942b377c5ec 人物專訪怎麼寫？訪綱、採訪、寫作心法 陳芷...,72.5
253,5b87af283aeadb001e5c167d 不正經古文講堂：流傳五千年的魯蛇哲學 聯經...,70.5
248,5b6c498c19937f001e0a8e4f 台灣小說名場面：五百字吸收文字內力 198...,67.8
59,56d68e0b182fba0e00c6d9d7 超濃郁！美味的食物插畫 藝術 繪畫與插畫...,64.2
40,568552e0e8ff9b20003cf518 角色設計1_人體骨架結構 遊戲美術工作者，...,64.2
87,5737f2acfad4ef0a006b6912 邊走邊畫，帶著手帳旅行去！ 插畫家與旅行作...,64.2
6,551a6be023774e0a001eb20c 療癒感手繪插畫食譜 很難明確的表達自己有多...,64.2
19,5591501e6dec460f00111314 動物插畫 吳振宇（Johnny） 1985...,64.2


In [137]:
users = pd.read_csv('../data/users.csv').fillna('')
users.iloc[50]['interests']

'手作_篆刻,手作_手工印刷,手作_刺繡,人文_文學,設計_動態設計,生活品味_壓力舒緩,攝影_後製剪輯,手作_手作小物,藝術_電腦繪圖,生活品味_更多生活品味,生活品味_心靈成長與教育,手作_手工書,生活品味_寵物,音樂_樂器,藝術_手寫字,藝術_字體設計,手作_更多手作,藝術_更多藝術,人文_社會科學,藝術_繪畫與插畫'

In [138]:
user_dict = {}
for user, title, interest, name in zip(users['user_id'], users['occupation_titles'],users['interests'],users['recreation_names']):
    user_dict[user] = ' '.join(title.split(',')) + ' ' + ' '.join(interest.split(',')) + ' ' + ' '.join(name.split(','))
list(user_dict.items())[:10]

[('54ccaa73a784960a00948687',
  ' 職場技能_創業 藝術_電腦繪圖 設計_介面設計 設計_動態設計 設計_平面設計 投資理財_投資觀念 行銷_數位行銷 藝術_角色設計 藝術_繪畫與插畫 職場技能_個人品牌經營 '),
 ('54dca4456d7d350900e86bae',
  ' 設計_動態設計 設計_平面設計 設計_應用設計 程式_程式入門 程式_程式語言 藝術_角色設計 設計_介面設計 程式_網頁前端 藝術_電腦繪圖 音樂_音樂創作 藝術_繪畫與插畫 '),
 ('54e421bac5c9c00900cd8d47',
  ' 設計_平面設計 職場技能_資料彙整 藝術_繪畫與插畫 行銷_數位行銷 職場技能_文書處理 職場技能_職場溝通 '),
 ('54e961d4c5c9c00900cd8d84',
  '金融業 投資理財_理財 攝影_影像創作 投資理財_投資觀念 藝術_更多藝術 音樂_樂器 投資理財_金融商品 音樂_音樂創作 '),
 ('54e9b744c5c9c00900cd8d8a',
  '資訊科技 法律、社會及文化專業 非營利組織 程式_網頁前端 投資理財_理財 投資理財_投資觀念 程式_程式語言 設計_設計理論 投資理財_金融商品 政治經濟 社會服務 舞台劇 電影'),
 ('54eac49bc5c9c00900cd8d95', ' 行銷_文案 投資理財_理財 投資理財_投資觀念 行銷_數位行銷 '),
 ('54ef3499b839040f0001607d',
  ' 攝影_商業攝影 設計_平面設計 藝術_電腦繪圖 攝影_影像創作 藝術_角色設計 藝術_繪畫與插畫 插畫 桌遊 棋類遊戲 素描 電影 電玩'),
 ('54ef34e2b839040f00016086', ' 職場技能_職場溝通 職場技能_資料彙整 行銷_數位行銷 職場技能_文書處理 '),
 ('54ef354db839040f0001608b',
  '藝文設計 自由業 音樂_人聲 藝術_電腦繪圖 音樂_音樂創作 程式_遊戲開發 藝術_字體設計 狗派 運動健身 電玩 電腦繪圖'),
 ('54ef3574b839040f0001608f',
  '藝文設計 出版業 自由業 職場技能_創業 職場技能_求職 職場技能_效率提升 程式_程式入門 職場

In [139]:
test_unseen = pd.read_csv('../data/test_unseen.csv')
with open('./submit_unseen_course.csv', 'w') as f:
    f.write('user_id,course_id\n')
    for i, user in  enumerate(test_unseen['user_id'][:]):
        t = []
        if(i % 500 == 0):
            print(i)
        for course in recommend_law(user_dict[user])['text']:
            
            t.append(course[:24])
        f.write('{},{}\n'.format(user, ' '.join(t)))

0
['經營', '分析', '旅行', '提升', '旅遊', '觀念', '投資', '量化', '商品', '理財', '程式', '金融', '製造業', '職場', '效率', '服務業', '技能', '個人', '品牌']
['插畫', '經營', '攝影', '品味', '語言', '電影', '繪畫', '色彩學', '健身', '前端', '創業', '手工', '生活', '政治', '賽事', '心靈', '旅行', '旅遊', '商業', '觀念', '創作', '投資', '書法', '網頁', '品牌', '料理', '網站', '貓派', '烹飪', '行銷', '理財', '印刷', '數位', '程式', '運動', '甜點', '經濟', '製造業', '職場', '狗派', '藝術', '設計', '金融', '人文', '影像', '社群', '服務業', '電玩', '電腦', '平面', '英文', '溝通', '技能', '音樂', '文學', '產品', '手作', '文案', '架設', '繪圖', '個人', '教育', '成長', '藝文', '閱讀', '親子']
['攝影', '電影', '交易', '分析', '前端', '創業', '賽事', '旅行', '旅遊', '創作', '投資', '量化', '商品', '網頁', '網站', '維護', '醫療', '資訊', '資料', '行銷', '理財', '程式', '數位', '金融', '運動', '職場', '後端', '理論', '影像', '軟體', '社群', '金融業', '開發', '技能', '架設', '科技', '閱讀', '科學']
['插畫', '品味', '繪畫', '小物', '字手作手作', '生活', '心靈', '手寫', '靈性', '字體', '行銷', '數位', '藝術', '設計', '角色', '電腦', '素描', '發展', '手作', '繪圖', '教育', '成長', '模型', '親子']
['語言', '融理', '分析', '思維', '旅行', '旅遊', '量化', '維護', '塊鏈', '健身金', '資訊', '安全', '理財', '程式', '運動', '軟體', '英文',

KeyboardInterrupt: 